# Tick data
For optimium results this notebook should be run during the Forex trading session

In [1]:
from ib_insync import *
util.startLoop()

ib=IB()
ib.connect('127.0.0.1', 1301, 15)

<IB connected to 127.0.0.1:1301 clientId=15>

## Streaming tick data
Create some forex contracts

In [2]:
contracts = [Forex(pair) for pair in (['EURUSD', 'USDJPY', 'GBPUSD', 'USDCHF', 'USDCAD', 'AUDUSD'])]
ib.qualifyContracts(*contracts)
eurusd = contracts[0]

Request streaming ticks for them:

In [3]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)

Wait a few seconds for the tickers to get filled

In [4]:
ticker = ib.ticker(eurusd)
ib.sleep(2)

ticker

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2020, 12, 28, 15, 42, 12, 591607, tzinfo=datetime.timezone.utc), bid=1.22173, bidSize=7000000, ask=1.22175, askSize=1080000, prevBid=1.22174, prevBidSize=3000000, prevAsk=1.22174, prevAskSize=1580000, high=1.2251, low=1.2181, close=1.2186)

The price of Forex ticks is always nan. To get a midpoint price use `midpoint()` or `marketPrice()`

The tickers are kept live updated. Try this a few times to see if the price changes

In [5]:
ticker.marketPrice()

1.22174

The following cell will start a 30 second loop that prints a live updated ticker table. It is updated on every tick change.

In [6]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(
        index=[c.pair() for c in contracts],
        columns = ['bidSize', 'bid', 'ask', 'askSize', 'high', 'low', 'close'])
    
def onPendingTickers(tickers):
    for t in tickers:
        df.loc[t.contract.pair()] = (
            t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
        clear_output(wait=True)
    display(df)
    
ib.pendingTickersEvent += onPendingTickers
ib.sleep(30)
ib.pendingTickersEvent -= onPendingTickers

,bidSize,bid,ask,askSize,high,low,close
EURUSD,3000000,1.2218,1.22181,1000000,1.2251,1.2181,1.2186
USDJPY,3000000,103.774,103.776,6000000,103.78,103.4,103.64
GBPUSD,3000000,1.34586,1.34591,1000000,1.3576,1.34565,1.3555
USDCHF,1000000,0.88921,0.88922,2000000,0.8919,0.8876,0.891
USDCAD,1000000,1.28603,1.28604,1000000,1.2878,1.28135,1.2875
AUDUSD,4000000,0.7571,0.75714,10000000,0.76225,0.75705,0.7591


New tick data is available in the `ticks` attribute of the pending tickers. The tick data will be cleared before the next update.

To stop the live subscriptions:

In [7]:
for contract in contracts:
    ib.cancelMktData(contract)

## Tick-by-Tick data
The ticks in the previous section are time-sampled by IB in order to cut on bandwidth. So with `reqMktData` not every tick from the exchange is sent. The promise of `reqTickByTickData` is to send every tick, just like the TWS Time & Sales window. This functionality is severly nerfed by a total of just three simultaneous subscriptions, where bid-ask ticks and sale ticks also use up a subscription each.

The tick-by-tick updates are available from `ticker.tickByTicks` and are signalled by `ib.pendingTickersEvent` or `ticker.updateEvent`.

In [8]:
ticker = ib.reqTickByTickData(eurusd, 'BidAsk')
ib.sleep(2)
print(ticker)

ib.cancelTickByTickData(ticker.contract, 'BidAsk')

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2020, 12, 28, 15, 42, 45, 177221, tzinfo=datetime.timezone.utc), bid=1.2218, bidSize=3000000, ask=1.22181, askSize=1000000, prevBid=1.22179, prevBidSize=1000000, prevAsk=1.2218, prevAskSize=4000000, high=1.2251, low=1.2181, close=1.2186)


## Historical tick data
Historical tick data can be fetched with a maximum of 1000 ticks at a time. Either the start time or the end time must be given, and one of them must remain empty.

In [12]:
import datetime

start = ''
end = datetime.datetime.now()
ticks = ib.reqHistoricalTicks(eurusd, start, end, 1000, 'BID_ASK', useRth=False)

ticks[-1]

HistoricalTickBidAsk(time=datetime.datetime(2020, 12, 28, 15, 46, 10, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(bidPastLow=False, askPastHigh=False), priceBid=1.22142, priceAsk=1.22145, sizeBid=11000000, sizeAsk=2000000)

In [13]:
ib.disconnect()